In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

# import oscar and clean data

In [2]:
oscar = pd.read_csv("../data/the_oscar_award.csv")
oscar.category = oscar.category.str.lower()
filtered = oscar[oscar.category.str.contains("(actor)|(actress)|(directing)", regex = True)]
filtered.loc[filtered.category.str.contains("(actor)|(actress)"), 'is_director'] = 0
filtered.loc[filtered.category.str.contains("directing"), 'is_director'] = 1

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [3]:
oscar_ppl = filtered[~filtered.name.str.contains("( and)|(,)|(and )", regex = True)]
oscar_ppl.is_director = oscar_ppl.is_director.astype(np.uint8)
oscar_ppl.head()

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
0,1927,1928,1,actor,Richard Barthelmess,The Noose,False,0
1,1927,1928,1,actor,Emil Jannings,The Last Command,True,0
2,1927,1928,1,actress,Louise Dresser,A Ship Comes In,False,0
3,1927,1928,1,actress,Janet Gaynor,7th Heaven,True,0
4,1927,1928,1,actress,Gloria Swanson,Sadie Thompson,False,0


In [4]:
oscar_ppl.shape

(2192, 8)

In [5]:
filtered[filtered.name.str.contains("( and)|(,)|(and )", regex = True)]

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
531,1937,1938,10,actor in a supporting role,Roland Young,Topper,False,0.0
3768,1961,1962,34,directing,"Robert Wise, Jerome Robbins",West Side Story,True,1.0
5655,1978,1979,51,directing,"Warren Beatty, Buck Henry",Heaven Can Wait,False,1.0
5942,1981,1982,54,actor in a supporting role,"Howard E. Rollins, Jr.",Ragtime,False,0.0
6043,1982,1983,55,actor in a supporting role,"Louis Gossett, Jr.",An Officer and a Gentleman,True,0.0
6289,1984,1985,57,directing,Roland Joffé,The Killing Fields,False,1.0
6506,1986,1987,59,directing,Roland Joffé,The Mission,False,1.0
7558,1996,1997,69,actor in a supporting role,"Cuba Gooding, Jr.",Jerry Maguire,True,0.0
8832,2007,2008,80,directing,Joel Coen and Ethan Coen,No Country for Old Men,True,1.0
9184,2010,2011,83,directing,Joel Coen and Ethan Coen,True Grit,False,1.0


In [6]:
#Manually modify the rows where there are special characters (for movie after 1986)
oscar_ppl = pd.concat([oscar_ppl, filtered[(filtered.index==7558) | (filtered.index==6506)]])
oscar_ppl.shape

(2194, 8)

In [7]:
oscar_ppl[oscar_ppl.name=="Joel Coen and Ethan Coen"]

,year_film,year_ceremony,ceremony,category,name,film,winner,is_director


In [8]:
oscar_ppl = pd.concat([oscar_ppl, filtered[(filtered.index==8832) | (filtered.index==9184)]], ignore_index=False)
oscar_ppl = pd.concat([oscar_ppl, filtered[(filtered.index==8832) | (filtered.index==9184)]], ignore_index=True)
oscar_ppl.shape

(2198, 8)

In [9]:
oscar_ppl[oscar_ppl.name== "Joel Coen and Ethan Coen"]

,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
2194,2007,2008,80,directing,Joel Coen and Ethan Coen,No Country for Old Men,True,1.0
2195,2010,2011,83,directing,Joel Coen and Ethan Coen,True Grit,False,1.0
2196,2007,2008,80,directing,Joel Coen and Ethan Coen,No Country for Old Men,True,1.0
2197,2010,2011,83,directing,Joel Coen and Ethan Coen,True Grit,False,1.0


In [10]:
oscar_ppl.at[2194, 'name'] = 'Joel Coen'
oscar_ppl.at[2195, 'name'] = 'Ethan Coen'
oscar_ppl.at[2196, 'name'] = 'Ethan Coen'
oscar_ppl.at[2197, 'name'] = 'Joel Coen'

In [11]:
oscar_ppl.shape

(2198, 8)

In [12]:
oscar_ppl.head()

,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
0,1927,1928,1,actor,Richard Barthelmess,The Noose,False,0.0
1,1927,1928,1,actor,Emil Jannings,The Last Command,True,0.0
2,1927,1928,1,actress,Louise Dresser,A Ship Comes In,False,0.0
3,1927,1928,1,actress,Janet Gaynor,7th Heaven,True,0.0
4,1927,1928,1,actress,Gloria Swanson,Sadie Thompson,False,0.0


In [18]:
oscar_ppl = oscar_ppl[oscar_ppl['year_film']>1985]
print('oscar award dataset has shape {}'.format(oscar_ppl.shape))
oscar.ppl.head()

oscar award dataset has shape (852, 8)


,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
1346,1986,1987,59,actor in a leading role,Dexter Gordon,'Round Midnight,False,0.0
1347,1986,1987,59,actor in a leading role,Bob Hoskins,Mona Lisa,False,0.0
1348,1986,1987,59,actor in a leading role,William Hurt,Children of a Lesser God,False,0.0
1349,1986,1987,59,actor in a leading role,Paul Newman,The Color of Money,True,0.0
1350,1986,1987,59,actor in a leading role,James Woods,Salvador,False,0.0


## movie industry

In [19]:
movie_industry = pd.read_csv("../data/movie_industry.csv", encoding = "ISO-8859-1" )
years = np.unique(movie_industry.year.values)
directors = np.unique(movie_industry.director.values)
actors = np.unique(movie_industry.star.values)
individuals = np.append(directors, actors)
years2 = np.repeat(years, len(individuals))
agents = np.tile(individuals, len(years))
is_director = np.tile(np.append(np.ones(len(directors)), np.zeros(len(actors))), len(years))
ID = np.tile(np.arange(len(individuals)), len(years))
time = np.repeat(np.arange(len(years)), len(individuals))


In [20]:
assert len(agents) == len(years2)
assert len(agents) == len(is_director)
assert len(agents) == len(ID)

In [22]:
df  = pd.DataFrame({"agent": agents,
                   "is_director": is_director,
                   "id": ID,
                   "year": years2,
                   "time": time})
print('There are {} people who double and actors and directors'.format(len(individuals)  - len(df.groupby(["agent"]).count())))
df

There are 171 people who double and actors and directors


,agent,is_director,id,year,time
0,A.R. Murugadoss,1.0,0,1986,0
1,Aamir Khan,1.0,1,1986,0
2,Aaron Blaise,1.0,2,1986,0
3,Aaron Harvey,1.0,3,1986,0
4,Aaron Kim Johnston,1.0,4,1986,0
...,...,...,...,...,...
163148,Zoey Deutch,0.0,5258,2016,30
163149,Zooey Deschanel,0.0,5259,2016,30
163150,Álex Angulo,0.0,5260,2016,30
163151,Élodie Bouchez,0.0,5261,2016,30


In [23]:
assert df.groupby("is_director").count().agent[1] == len(directors)*len(years)
assert len(df.groupby(["agent", "is_director"]).count()) == len(individuals)

In [27]:
%%time
total_nominations =[]
for i in df.index:
    if df.is_director[i]:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == True)]
        temp = temp[temp.year_film <= df.year[i]]
        nomination = temp.shape[0]
        total_nominations.append(nomination)
    else:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == False)]
        temp = temp[temp.year_film <= df.year[i]]
        nomination = temp.shape[0]
        total_nominations.append(nomination)

CPU times: user 3min 12s, sys: 669 ms, total: 3min 13s
Wall time: 3min 13s


In [28]:
df["nominations"] = total_nominations
df.head()

,agent,is_director,id,year,time,nominations
0,A.R. Murugadoss,1.0,0,1986,0,0
1,Aamir Khan,1.0,1,1986,0,0
2,Aaron Blaise,1.0,2,1986,0,0
3,Aaron Harvey,1.0,3,1986,0,0
4,Aaron Kim Johnston,1.0,4,1986,0,0


In [31]:
%%time
total_win_counts=[]
for i in df.index:
    if df.is_director[i]:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == True)]
        temp = temp[temp.year_film <= df.year[i]]
        win = sum(temp.winner.values)
        total_win_counts.append(win)
    else:
        temp = oscar_ppl[(oscar_ppl.name == df.agent[i]) & (oscar_ppl.is_director == False)]
        temp = temp[temp.year_film <= df.year[i]]
        win = sum(temp.winner.values)
        total_win_counts.append(win)

CPU times: user 3min 36s, sys: 985 ms, total: 3min 37s
Wall time: 3min 38s


In [32]:
df["win_counts"] = total_win_counts
df.head()

,agent,is_director,id,year,time,nominations,win_counts
0,A.R. Murugadoss,1.0,0,1986,0,0,0
1,Aamir Khan,1.0,1,1986,0,0,0
2,Aaron Blaise,1.0,2,1986,0,0,0
3,Aaron Harvey,1.0,3,1986,0,0,0
4,Aaron Kim Johnston,1.0,4,1986,0,0,0


## sanity check

In [35]:
df[df.agent == "Clint Eastwood"].head(50)

,agent,is_director,id,year,time,nominations,win_counts
459,Clint Eastwood,1.0,459,1986,0,0,0
3201,Clint Eastwood,0.0,3201,1986,0,0,0
5722,Clint Eastwood,1.0,459,1987,1,0,0
8464,Clint Eastwood,0.0,3201,1987,1,0,0
10985,Clint Eastwood,1.0,459,1988,2,0,0
13727,Clint Eastwood,0.0,3201,1988,2,0,0
16248,Clint Eastwood,1.0,459,1989,3,0,0
18990,Clint Eastwood,0.0,3201,1989,3,0,0
21511,Clint Eastwood,1.0,459,1990,4,0,0
24253,Clint Eastwood,0.0,3201,1990,4,0,0


In [36]:
oscar_ppl[oscar_ppl.name == "Clint Eastwood"].head(50)

,year_film,year_ceremony,ceremony,category,name,film,winner,is_director
1496,1992,1993,65,actor in a leading role,Clint Eastwood,Unforgiven,False,0.0
1519,1992,1993,65,directing,Clint Eastwood,Unforgiven,True,1.0
1793,2003,2004,76,directing,Clint Eastwood,Mystic River,False,1.0
1797,2004,2005,77,actor in a leading role,Clint Eastwood,Million Dollar Baby,False,0.0
1815,2004,2005,77,directing,Clint Eastwood,Million Dollar Baby,True,1.0
1866,2006,2007,79,directing,Clint Eastwood,Letters from Iwo Jima,False,1.0
